# Exploration

## Objectif
L'objectif est de permettre le suivi régulier des délais de règlement des factures :
- au niveau Entreprise, les factures émises par l'entreprise
- au niveau Expert Comptable, les factures émises par les entreprises suivies par le cabinet
- au niveau RCA, dégager des tendances

![image](../img/schema.png)

*Centralise ici les imports python dont tu as besoin :*

In [258]:
import pandas as pd
import numpy as np
from validate_email import validate_email
import re

# 1. Datasets : chargement et vérification

## 1.1 Dataset Accountant

### Charge un dataframe nommé `accountant_df` avec les données du fichier `data/Accountant.xlsx`

In [259]:
accountant_df = pd.read_excel(r"../data/Accountant.xlsx", dtype={"id":  np.int32})

### Affiche les informations pertinentes ou applique les vérifications nécessaires selon toi sur le dataframe `accountant_df`

In [260]:
# on vérifie que le nom et le prénom ne contiennent pas de chiffre
accountant_df = accountant_df[accountant_df['first_name'].apply(
    lambda x: bool(re.search(r"^[a-zA-Z '-]*$", str(x))))]
accountant_df = accountant_df[accountant_df['last_name'].apply(
    lambda x: bool(re.search(r"^[a-zA-Z '-]*$", str(x))))]

#  on vérifie la validité de l'email
accountant_df = accountant_df[accountant_df['email'].apply(validate_email)]

accountant_df["accountant_name"] = accountant_df["first_name"] + " " +  accountant_df["last_name"]
accountant_df = accountant_df.drop(["first_name", "last_name"], axis=1)

accountant_df

,id,email,accountant_name
0,1,nborrington0@hubpages.com,Nada Borrington
1,2,ahendrik1@yolasite.com,Anetta Hendrik
2,3,cstiller2@ca.gov,Catarina Stiller
3,4,fevered3@indiatimes.com,Freedman Evered
4,5,gwatts4@printfriendly.com,Giuseppe Watts
5,6,hborrie5@nydailynews.com,Hannah Borrie
6,7,imacintyre6@tinyurl.com,Ingeberg MacIntyre
7,8,eilyukhov7@simplemachines.org,Emelen Ilyukhov
8,9,glevens8@microsoft.com,Gerhardt Levens
9,10,kligoe9@topsy.com,Kenneth Ligoe


## 1.2 Dataset Company

### Charge un dataframe nommé `company_df` avec les données du fichier `data/Company.xlsx`

In [261]:
company_df = pd.read_excel(r"../data/Company.xlsx", dtype={"id": np.int32, "company_accountant": np.int32})


### Affiche les informations pertinentes ou applique les vérifications nécessaires selon toi sur le dataframe `company_df`

In [262]:
# une premiere vérification pertinente serait de verifier que les valeurs de la colonne
# company_area appartiennent a une liste de valeure définie. Cela permettrait d'eviter les doublons
# comme par example "Real Estate" vs "real_estate" etc
# Normalement l'utilisateur aurait probablement a choisir une valeure via une combo box pour eviter 
# ce genre de probleme
lpossible_area = [
    "Computer Software: Prepackaged Software",
    "Environmental Services",
    "Major Banks",
    "Major Chemicals",
    "Major Pharmaceuticals",
    "Real Estate",
    "Semiconductors", 
    "Telecommunications Equipment",
    np.nan,
    ]
company_df = company_df[company_df['company_area'].apply(lambda x: x in lpossible_area)]

# pour notre problématique les colonnes company_address et company_city ne semblent pas pertinente et 
# peuvent etre retirée
company_df = company_df.drop(["company_address", "company_city", "company_area"], axis=1)

company_df

,id,company_name,company_accountant
0,1,"Larson, Hilpert and Hegmann",3
1,2,Davis-Brekke,5
2,3,"Hirthe, Lesch and Schroeder",5
3,4,"Prosacco, Brakus and Bayer",2
4,5,Rippin and Sons,6
5,6,Collins-Mitchell,7
6,7,Wolf-Bartell,4
7,8,Auer-Robel,1
8,9,DuBuque and Sons,10
9,10,Botsford-Boyle,9


## 1.3 Dataset Customer

### Charge un dataframe nommé `customer_df` avec les données du fichier `data/Customer.xlsx`

In [263]:
customer_df = pd.read_excel(r"../data/Customer.xlsx", dtype={"id":  np.int32})

### Affiche les informations pertinentes ou applique les vérifications nécessaires selon toi sur le dataframe `customer_df`

In [264]:
# Les memes vérifications peuvent s'appliquer ici sur le nom (pas de chiffre), sur l'adresse
# (chiffres + lettres) ou sur la ville (que des lettres)
customer_df = customer_df[customer_df['customer_name'].apply(
    lambda x: bool(re.search(r"^[a-zA-Z '-]*$", str(x))))]

# pour notre problématique les colonnes customer_address et customer_city ne semblent pas pertinente et 
# peuvent etre retirée
customer_df = customer_df.drop(["customer_address", "customer_city"], axis=1)

customer_df


,id,customer_name
0,1,Elyssa Masarrat
1,2,Leone MacFadin
2,3,Gerda Norton
3,4,Myrna Mulhall
4,5,Greta Cone
...,...,...
65,66,Ava Getty
66,67,Avrom McColl
67,68,Margery Oswick
68,69,Wain Tidbald


## 1.4 Dataset Invoice

### Charge un dataframe nommé `invoice_df` avec les données du fichier `data/Invoice.xlsx`

In [273]:
invoice_df = pd.read_excel(r"../data/Invoice.xlsx", 
                           dtype={"id": np.int32, "invoice_company": np.int32, "invoice_customer": np.int32})

### Affiche les informations pertinentes ou applique les vérifications nécessaires selon toi sur le dataframe `invoice_df`

In [274]:
# on peut vérifier que les sommes demandés sont corrects, cad un float positif
invoice_df = invoice_df[invoice_df['invoice_amount'].apply(lambda x: x > 0)]
invoice_df = invoice_df[invoice_df['invoice_payment_date'].notnull()]

invoice_df = invoice_df.drop(["invoice_amount", "invoice_currency"], axis=1)

invoice_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 689 entries, 0 to 998
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     689 non-null    int32         
 1   invoice_creation_date  689 non-null    datetime64[ns]
 2   invoice_payment_date   689 non-null    datetime64[ns]
 3   invoice_company        689 non-null    int32         
 4   invoice_customer       689 non-null    int32         
dtypes: datetime64[ns](2), int32(3)
memory usage: 24.2 KB


# 2. Fusion des datasets

### Crée un dataframe nommé `dataset_df` à partir des datasets préalablement chargés

In [275]:
# merge table comptable et company
merge1_df = pd.merge(company_df, accountant_df, left_on="company_accountant", right_on="id").drop(
    ["company_accountant", "id_y"], axis=1)

# merge table invoice et cutomer 
merge2_df = pd.merge(invoice_df, customer_df, left_on="invoice_customer", right_on="id").drop(
    ["invoice_customer", "id_y", "id_x"], axis=1)

dataset_df = pd.merge(merge1_df, merge2_df, left_on="id_x", right_on="invoice_company").drop(
    ["invoice_company", "id_x"], axis=1)

dataset_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 689 entries, 0 to 688
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   company_name           689 non-null    object        
 1   email                  689 non-null    object        
 2   accountant_name        689 non-null    object        
 3   invoice_creation_date  689 non-null    datetime64[ns]
 4   invoice_payment_date   689 non-null    datetime64[ns]
 5   customer_name          689 non-null    object        
dtypes: datetime64[ns](2), object(4)
memory usage: 37.7+ KB


### Sauvegarde le dataframe `dataset_df` au format au chemin suivant `data/dataset.csv`

In [268]:
dataset_df.to_csv(r"../data/dataset.csv", index=False)

# 3. Enregistre ton notebook !
